# exp081
derivedデータを使ったbaselineの復元

In [1]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score
pd.set_option('display.max_rows', 100)
from math import * 
import scipy.optimize as opt
import multiprocessing

In [2]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [3]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [4]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [5]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [6]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)

In [7]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [8]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [9]:
class train_result:
    def __init__(self, df):
        self.df = df
        self.gt = get_ground_truth()
        self.bl = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
        
        self.gt = self.gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
        self.df = self.df.merge(self.gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
        self.df['phone'] = self.df['collectionName'] + '_' + self.df['phoneName']
        self.df['err'] =  calc_haversine(self.df['latDeg_gt'], self.df['lngDeg_gt'], self.df['latDeg'], self.df['lngDeg'])
        
        self.phone_res = self.calc_err('phone')
        self.clc_res = self.calc_err('collectionName')
        self.phonename_res = self.calc_err('phoneName')
        
    def calc_err(self, by):
        res = self.df.groupby(by)['err'].agg([percentile50, percentile95])
        res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2
        return res
    
    @property
    def score(self):
        return self.phone_res['p50_p90_mean'].mean()
    @property
    def raw_data(self):
        return self.df
    @property
    def err(self):
        return self.phone_res
    @property
    def collection_err(self):
        return self.clc_res
    @property
    def phonename_err(self):
        return self.phonename_res
    
    def viz_map(self, collection, show_gt=True, show_bl=True):
        tmp = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp2 = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg_gt', 'lngDeg_gt']]
        tmp2 = tmp2.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'})
        tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
        tmp3 = self.bl[self.bl['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp3['phoneName'] = tmp3['phoneName'] + '_BL'
        
        if show_gt:
            tmp = tmp.append(tmp2)
        if show_bl:
            tmp = tmp.append(tmp3)
        visualize_collection(tmp, collection)

In [10]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = pd.read_csv(INPUT + '/prep/ground_truth_train.csv')
    return base_train, base_test, sample_sub, ground_truth

In [11]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

# baselineの再作成

In [12]:
def prepare_calc_baseline(df):
    light_speed = 299_792_458
    omega_e = 7.2921151467e-5
    
    # Corrected pseudorange according to data instructions
    df['correctedPrM'] = df['rawPrM'] + \
                         df['satClkBiasM'] - \
                         df['isrbM'] - \
                         df['ionoDelayM'] - \
                         df['tropoDelayM']
    
    # Time it took for signal to travel
    df['transmissionTimeSeconds'] = df['correctedPrM'] / light_speed
    
    # Compute true sat positions at arrival time
    df['xSatPosMRotated'] = \
        np.cos(omega_e * df['transmissionTimeSeconds']) * df['xSatPosM'] \
        + np.sin(omega_e * df['transmissionTimeSeconds']) * df['ySatPosM']

    df['ySatPosMRotated'] = \
        - np.sin(omega_e * df['transmissionTimeSeconds']) * df['xSatPosM'] \
        + np.cos(omega_e * df['transmissionTimeSeconds']) * df['ySatPosM']

    df['zSatPosMRotated'] = df['zSatPosM']
    
    # Uncertainty weight for the WLS method
    df['uncertaintyWeight'] = 1 / df['rawPrUncM']
    
    return df

In [13]:
def calc_baseline_point(df):

    def distance(sat_pos, x):
        sat_pos_diff = sat_pos.copy(deep=True)

        sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
        sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
        sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]

        sat_pos_diff['d'] = sat_pos_diff['uncertaintyWeight'] * \
                            (np.sqrt((sat_pos_diff['xSatPosMRotated']**2 + sat_pos_diff['ySatPosMRotated']**2 + sat_pos_diff['zSatPosMRotated']**2)) + \
                             x[3] - sat_pos_diff['correctedPrM'])

        return sat_pos_diff['d']

    def distance_fixed_satpos(x):
        return distance(df[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)
    
    x0 = [0,0,0,0]
    opt_res = opt.least_squares(distance_fixed_satpos, x0)
    # Optimiser yields a position in the ECEF coordinates
    opt_res_pos = opt_res.x
    
    # ECEF position to lat/long
    wls_estimated_pos = ecef2lla(*opt_res_pos[:3])
    wls_estimated_pos = np.squeeze(wls_estimated_pos)
    
    return wls_estimated_pos[0], wls_estimated_pos[1]

In [14]:
def get_derived_data(train_test, collection, phonename):
    derived = pd.read_csv(INPUT + f'/{train_test}/{collection}/{phonename}/{phonename}_derived.csv')
    raw = pd.read_csv(INPUT + f'/prep/gnss/{train_test}/{collection}/{phonename}/Raw.csv')
    
    # Assume we've loaded a dataframe from _GnssLog.txt for only lines beginning with "Raw", we denote this df_raw. Next, assume we've loaded a dataframe from _derived.csv. We denote this df_derived.

    # Create a new column in df_raw that corresponds to df_derived['MillisSinceGpsEpoch']
    raw['millisSinceGpsEpoch'] = np.floor( (raw['TimeNanos'] - raw['FullBiasNanos']) / 1000000.0).astype(int)
    
    # Change each value in df_derived['MillisSinceGpsEpoch'] to be the prior epoch.
    raw_timestamps = raw['millisSinceGpsEpoch'].unique()
    derived_timestamps = derived['millisSinceGpsEpoch'].unique()

    # The timestamps in derived are one epoch ahead. We need to map each epoch
    # in derived to the prior one (in Raw).
    indexes = np.searchsorted(raw_timestamps, derived_timestamps)
    from_t_to_fix_derived = dict(zip(derived_timestamps, raw_timestamps[indexes-1]))
    derived['millisSinceGpsEpoch'] = np.array(list(map(lambda v: from_t_to_fix_derived[v], derived['millisSinceGpsEpoch'])))

    # Compute signal_type in df_raw.
    # Map from constellation id to frequencies and signals.
    CONSTEL_FREQ_TABLE = {
        0: {'UNKNOWN': (0, 999999999999)},
        1: {
            'GPS_L1': (1563000000, 1587000000),
            'GPS_L2': (1215000000, 1240000000),
            'GPS_L5': (1164000000, 1189000000)
        },
        3: {
            'GLO_G1': (1593000000, 1610000000),
            'GLO_G2': (1237000000, 1254000000)
        },
        4: {
            'QZS_J1': (1563000000, 1587000000),
            'QZS_J2': (1215000000, 1240000000),
            'QZS_J5': (1164000000, 1189000000)
        },
        5: {
            'BDS_B1C': (1569000000, 1583000000),
            'BDS_B1I': (1553000000, 1568990000),
            'BDS_B2A': (1164000000, 1189000000),
            'BDS_B2B': (1189000000, 1225000000)
        },
        6: {
            'GAL_E1': (1559000000, 1591000000),
            'GAL_E5A': (1164000000, 1189000000),
            'GAL_E5B': (1189000000, 1218000000),
            'GAL_E6': (1258000000, 1300000000)
        },
        7: {
            'IRN_S': (2472000000, 2512000000),
            'IRN_L5': (1164000000, 1189000000)
        },
    }

    def SignalTypeFromConstellationAndFequency(constel, freq_hz):
        'Returns the signal type as a string for the given constellation and frequency.'
        freqs = CONSTEL_FREQ_TABLE.get(constel, {})
        for id_freq_range in freqs.items():
            rng = id_freq_range[1]
            if rng[0] <= freq_hz <= rng[1]:
                return id_freq_range[0]
        return 'UNKNOWN'

    signal_types = itertools.chain(*[c.keys() for c in CONSTEL_FREQ_TABLE.values()])
    sig_type_cat = pd.api.types.CategoricalDtype(categories=signal_types)
    raw['SignalType'] = raw.apply(lambda r: SignalTypeFromConstellationAndFequency(r.ConstellationType, r.CarrierFrequencyHz), axis=1).astype(sig_type_cat)

    # Fix QZS Svids issue. 

    # The SVID of any QZS sat in derived may be changed. Since it may be a many to one relationship, we'll need to adjust the values in Raw.
    new_to_old = {1:(183, 193), 2:(184, 194, 196), 3:(187, 189, 197, 199), 4:(185, 195, 200)}
    # Maps original svid to new svid for only ConstellationType=4.
    old_to_new={}
    for new_svid, old_svids in new_to_old.items():
        for s in old_svids:
            old_to_new[s] = new_svid
    raw['svid'] = raw.apply(lambda r: old_to_new.get(r.Svid, r.Svid) if r.ConstellationType == 4 else r.Svid, axis=1)
    del raw['collectionName']
    del raw['phoneName']

    derived = derived.merge(raw, on=['millisSinceGpsEpoch', 'svid'], how='left')
    return derived

In [15]:
def calc_baseline(args):
    s_th = 1
    
    phone, df = args
    collection = phone.split('_')[0]
    phonename = phone.split('_')[1]
    derived = get_derived_data('train', collection, phonename)
    derived = prepare_calc_baseline(derived)
    
    idx = list(df.index)
    s_list = []
    n_list = []
    lat_list = []
    lng_list = []
    unc_mean_list = []
    unc_max_list = []
    
    for j,i in enumerate(idx):
        s = df.at[i, 'millisSinceGpsEpoch']
        tmp = derived[derived['millisSinceGpsEpoch']==s].copy()
        n = len(tmp)
        s_list.append(s)
        n_list.append(n)
        
        if n == 0:    
            lat_list.append(np.nan)
            lng_list.append(np.nan)
            unc_mean_list.append(np.nan)
            unc_max_list.append(np.nan)        
        
        else:
            res = calc_baseline_point(tmp)
            lat_list.append(res[0])
            lng_list.append(res[1])
            unc_mean_list.append(tmp['uncertaintyWeight'].mean())
            unc_max_list.append(tmp['uncertaintyWeight'].max())
    
    output_df = pd.DataFrame()
    output_df['millisSinceGpsEpoch'] = s_list
    output_df['latDeg'] = lat_list
    output_df['lngDeg'] = lng_list
    output_df['n'] = n_list
    output_df['unc_mean'] = unc_mean_list
    output_df['unc_max'] = unc_max_list
    output_df['collectionName'] = collection
    output_df['phoneName'] = phonename
    output_df['phone'] = phone
    
    return output_df

In [16]:
train, test, sub, gt = get_data()

In [17]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(calc_baseline, train.groupby('phone'))
    dfs = tqdm(dfs)
    dfs = list(dfs)
result = pd.concat(dfs)

0it [00:00, ?it/s]

In [18]:
train_tmp = train[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']].copy()
train_tmp.columns = ['phone', 'millisSinceGpsEpoch', 'latDeg_bl', 'lngDeg_bl']
result = result.merge(train_tmp, on=['phone', 'millisSinceGpsEpoch'], how='left')

In [19]:
gt['phone'] = gt['collectionName'] + '_' + gt['phoneName']
gt_tmp = gt[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']].copy()
gt_tmp.columns = ['phone', 'millisSinceGpsEpoch', 'latDeg_gt', 'lngDeg_gt']
result = result.merge(gt_tmp, on=['phone', 'millisSinceGpsEpoch'], how='left')

In [20]:
result['rb_bl_err'] = calc_haversine(result['latDeg_bl'], result['lngDeg_bl'], result['latDeg'], result['lngDeg'])
result['rb_gt_err'] = calc_haversine(result['latDeg_gt'], result['lngDeg_gt'], result['latDeg'], result['lngDeg'])
result['bl_gt_err'] = calc_haversine(result['latDeg_gt'], result['lngDeg_gt'], result['latDeg_bl'], result['lngDeg_bl'])

In [21]:
result.dropna()

,millisSinceGpsEpoch,latDeg,lngDeg,n,unc_mean,unc_max,collectionName,phoneName,phone,latDeg_bl,lngDeg_bl,latDeg_gt,lngDeg_gt,rb_bl_err,rb_gt_err,bl_gt_err
25677,1273538932663,37.690738,-122.392123,41,0.275137,0.834028,2020-05-14-US-MTV-2,Pixel4XLModded,2020-05-14-US-MTV-2_Pixel4XLModded,37.690731,-122.392110,37.690734,-122.392080,1.296210,3.780331,2.695096
25679,1273538934642,37.690772,-122.391906,43,0.259539,0.667111,2020-05-14-US-MTV-2,Pixel4XLModded,2020-05-14-US-MTV-2_Pixel4XLModded,37.690757,-122.391909,37.690792,-122.391921,1.719387,2.534406,4.036859
25680,1273538935657,37.690810,-122.391794,46,0.305518,0.555864,2020-05-14-US-MTV-2,Pixel4XLModded,2020-05-14-US-MTV-2_Pixel4XLModded,37.690793,-122.391799,37.690794,-122.391830,2.014880,3.685821,2.764286
25681,1273538936652,37.690832,-122.391775,42,0.300737,0.834028,2020-05-14-US-MTV-2,Pixel4XLModded,2020-05-14-US-MTV-2_Pixel4XLModded,37.690836,-122.391778,37.690776,-122.391739,0.511616,6.917315,7.428683
25683,1273538938655,37.690674,-122.391539,45,0.248427,0.667111,2020-05-14-US-MTV-2,Pixel4XLModded,2020-05-14-US-MTV-2_Pixel4XLModded,37.690697,-122.391556,37.690689,-122.391563,2.963600,2.736130,1.029373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131337,1303760272435,37.334486,-121.899629,58,0.302766,0.834028,2021-04-29-US-SJC-2,Pixel4,2021-04-29-US-SJC-2_Pixel4,37.334480,-121.899635,37.334473,-121.899612,0.795952,2.136721,2.210786
131338,1303760273435,37.334470,-121.899634,61,0.291496,0.834028,2021-04-29-US-SJC-2,Pixel4,2021-04-29-US-SJC-2_Pixel4,37.334480,-121.899634,37.334473,-121.899612,1.181159,2.041997,2.178279
131339,1303760274435,37.334495,-121.899624,59,0.299494,0.834028,2021-04-29-US-SJC-2,Pixel4,2021-04-29-US-SJC-2_Pixel4,37.334507,-121.899614,37.334473,-121.899612,1.584296,2.693788,3.770287
131340,1303760275435,37.334481,-121.899612,52,0.269677,0.555864,2021-04-29-US-SJC-2,Pixel4,2021-04-29-US-SJC-2_Pixel4,37.334460,-121.899611,37.334473,-121.899612,2.231074,0.873321,1.358132


In [22]:
result_grouped = result.dropna().groupby('phone')[['rb_bl_err', 'rb_gt_err', 'bl_gt_err']].mean().reset_index()
result_grouped

,phone,rb_bl_err,rb_gt_err,bl_gt_err
0,2020-05-14-US-MTV-1_Pixel4,0.418768,1.482876,1.456455
1,2020-05-14-US-MTV-2_Pixel4,0.575532,1.506461,1.494108
2,2020-05-14-US-MTV-2_Pixel4XLModded,2.068235,4.400295,3.976445
3,2020-05-21-US-MTV-1_Pixel4,0.759120,2.364323,2.152806
4,2020-05-21-US-MTV-2_Pixel4,0.426356,1.289660,1.297878
5,2020-05-29-US-MTV-1_Pixel4,6.213374,9.705796,5.071355
6,2020-05-29-US-MTV-1_Pixel4XLModded,0.514790,2.367590,2.405069
7,2020-05-29-US-MTV-2_Pixel4,0.241709,2.515694,2.471753
8,2020-05-29-US-MTV-2_Pixel4XL,0.213564,2.608899,2.583133
9,2020-06-04-US-MTV-1_Pixel4,0.535345,1.895411,1.877974


In [23]:
result.to_csv(OUTPUT + '/result.csv', index=False)
result_grouped.to_csv(OUTPUT + '/result_grouped.csv', index=False)

In [24]:
result[result['phone']=='2020-05-14-US-MTV-1_Pixel4'].sort_values('millisSinceGpsEpoch').head(100)

,millisSinceGpsEpoch,latDeg,lngDeg,n,unc_mean,unc_max,collectionName,phoneName,phone,latDeg_bl,lngDeg_bl,latDeg_gt,lngDeg_gt,rb_bl_err,rb_gt_err,bl_gt_err
62818,1273529463442,37.423580,-122.094102,56,0.431238,1.666667,2020-05-14-US-MTV-1,Pixel4,2020-05-14-US-MTV-1_Pixel4,37.423575,-122.094091,37.423576,-122.094132,1.059345,2.676633,3.586842
62819,1273529464442,37.423577,-122.094099,58,0.458107,1.666667,2020-05-14-US-MTV-1,Pixel4,2020-05-14-US-MTV-1_Pixel4,37.423578,-122.094101,37.423576,-122.094132,0.195575,2.936807,2.745901
62820,1273529465442,37.423579,-122.094116,60,0.461474,1.666667,2020-05-14-US-MTV-1,Pixel4,2020-05-14-US-MTV-1_Pixel4,37.423573,-122.094111,37.423576,-122.094132,0.830275,1.422621,1.888409
62821,1273529466442,37.423580,-122.094121,60,0.469393,1.666667,2020-05-14-US-MTV-1,Pixel4,2020-05-14-US-MTV-1_Pixel4,37.423583,-122.094121,37.423576,-122.094132,0.257907,1.121097,1.213483
62822,1273529467442,37.423578,-122.094116,54,0.501733,1.666667,2020-05-14-US-MTV-1,Pixel4,2020-05-14-US-MTV-1_Pixel4,37.423579,-122.094114,37.423576,-122.094132,0.269002,1.412495,1.650722
62823,1273529468442,37.423583,-122.094126,59,0.491166,1.666667,2020-05-14-US-MTV-1,Pixel4,2020-05-14-US-MTV-1_Pixel4,37.423578,-122.094126,37.423576,-122.094132,0.498605,0.912552,0.557491
62824,1273529469442,37.423574,-122.094114,62,0.481905,1.666667,2020-05-14-US-MTV-1,Pixel4,2020-05-14-US-MTV-1_Pixel4,37.423572,-122.094114,37.423576,-122.094132,0.143622,1.581372,1.649859
62825,1273529470442,37.423564,-122.094108,56,0.465226,1.666667,2020-05-14-US-MTV-1,Pixel4,2020-05-14-US-MTV-1_Pixel4,37.423568,-122.094116,37.423576,-122.094132,0.860865,2.490695,1.629857
62826,1273529471442,37.423575,-122.094110,60,0.479054,1.666667,2020-05-14-US-MTV-1,Pixel4,2020-05-14-US-MTV-1_Pixel4,37.423581,-122.094115,37.423576,-122.094132,0.878784,1.973388,1.603412
62827,1273529472442,37.423592,-122.094124,60,0.486664,1.666667,2020-05-14-US-MTV-1,Pixel4,2020-05-14-US-MTV-1_Pixel4,37.423594,-122.094123,37.423576,-122.094132,0.212873,1.927670,2.140313
